<a href="https://colab.research.google.com/github/ilgiz-n/exupery/blob/main/LP_formwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pulp import *

A = [5650, 3800, 5800, 4450]

# Создание задачи линейного программирования
prob = LpProblem("Formwork_Problem", LpMinimize)
delta = LpVariable(name="delta", lowBound=0, cat="Integer")

# Описываем переменные
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat="Integer") for i in range(1, 7)}

# Описываем целевую функцию
prob += 1550 * x[1] + 1700 * x[2] + 1775 * x[3] + 1850 * x[4] + 2250 * x[5] + 2450 * x[6]

# Описываем ограничения

prob += 300 * x[1] + 400 * x[2] + 450 * x[3] + 500 * x[4] + 750 * x[5] + 900 * x[6] == A[0] + delta
prob += delta <= 250

# Решаем задачу
results = prob.solve()

# Выводим результаты
print(f"status: {prob.status}, {LpStatus[prob.status]}")
print(f"Total cost: {prob.objective.value() * 2}")

for var in prob.variables():
    print(f"{var.name}: {var.value() * 2}")

status: 1, Optimal
Total cost: 32400.0
delta: 0.0
x1: 0.0
x2: 2.0
x3: 0.0
x4: 0.0
x5: 2.0
x6: 10.0


In [3]:
from pulp import *
import pandas as pd

# Входные параметры
# Длины стен в рабочей зоне

# Опции с угловыми элементами (алюминий и сталь, соответственно):
y1 = 300
y2 = 250

# Длина стен в зоне 'A' с плана:
walls_A = [5850, 4300, 6000, 4650]

# Рассчитываем откорректированные длины для опции y2 

d1A_y2 = walls_A[0] - y2 + 50
d2A_y2 = walls_A[1] - 2 * y2
d3A_y2 = walls_A[2] - y2 + 50
d4A_y2 = walls_A[3] - y2 + 50

# Получаем откорректированные длины
A = [d1A_y2, d2A_y2, d3A_y2, d4A_y2] 

# Данные об опалубке в формате "ширина:стоимость"
cost = {
    300: 1550, 
    400: 1700, 
    450: 1775, 
    500: 1850, 
    750: 2250, 
    900: 2450
}

# Решаем задачу линейного программирования для каждого значения A[i] из списка A
result_dict = {}
total_cost = 0
for i in range(len(A)):
    # Создаем задачу линейного программирования
    prob = LpProblem(f"Formwork_Problem{i}", LpMinimize)
    # Описываем переменные
    x = {i: LpVariable(name=f"{[*cost][i-1]}", 
                       lowBound=0, cat="Integer") for i in range(1,len(cost)+1)}
    delta = LpVariable(name="delta", lowBound=0, cat="Integer")
    # Определяем целевую функцию
    prob += (
        lpSum([*cost.values()][i-1] * x[i] for i in x),
        "Total Cost of panels for a wall",
    )
    # Добавляем ограничения
    prob += (
        lpSum([*cost][i-1] * x[i] for i in x) == A[i] + delta,
        "Total width constraint of panels for a wall [i]",
    )
    prob += delta <= 250
    # Решаем задачу
    results = prob.solve()
    # Добавляем результаты в словарь
    total_cost += prob.objective.value()
    result_dict[A[i]] = {}
    # Умножаем на 2, для подсчета с двух сторон
    result_dict[A[i]]['Cost'] = prob.objective.value() * 2
    for var in prob.variables():
        result_dict[A[i]][var.name] = var.value() * 2
        

# Выводим результаты в виде таблицы
result_table = pd.DataFrame(result_dict).T
result_table.index.name = 'Wall'
result_table.loc['Total']= result_table.sum()
print("Результаты для зоны A\n")
print(result_table)

Результаты для зоны A

           Cost  300  400  450  500  750   900  delta
Wall                                                 
5650    32400.0  0.0  2.0  0.0  0.0  2.0  10.0    0.0
3800    22500.0  0.0  0.0  0.0  2.0  4.0   4.0    0.0
5800    32800.0  0.0  2.0  0.0  0.0  0.0  12.0    0.0
4450    24500.0  0.0  0.0  0.0  0.0  0.0  10.0  100.0
Total  112200.0  0.0  4.0  0.0  2.0  6.0  36.0  100.0


In [4]:
from pulp import *
import pandas as pd

# Входные параметры
# Длины стен в рабочей зоне

# Опции с угловыми элементами (алюминий и сталь, соответственно):
y1 = 300
y2 = 250

# Длина стен в зоне 'A' с плана:
walls_B = [5400, 4800, 4600, 6000]

# Получаем откорректированные длины для опции y2 
B = [x - y2 + 50 for x in walls_B]

# Данные об опалубке в формате "ширина:стоимость"
cost = {
    300: 1550, 
    400: 1700, 
    450: 1775, 
    500: 1850, 
    750: 2250, 
    900: 2450
}

# Решаем задачу линейного программирования для каждого значения A[i] из списка A
result_dict = {}
total_cost = 0
for i in range(len(B)):
    # Создаем задачу линейного программирования
    prob = LpProblem(f"Formwork_Problem{i}", LpMinimize)
    # Описываем переменные
    x = {i: LpVariable(name=f"{[*cost][i-1]}", 
                       lowBound=0, cat="Integer") for i in range(1,len(cost)+1)}
    delta = LpVariable(name="delta", lowBound=0, cat="Integer")
    # Определяем целевую функцию
    prob += (
        lpSum([*cost.values()][i-1] * x[i] for i in x),
        "Total Cost of panels for a wall",
    )
    # Добавляем ограничения
    prob += delta <= 250
    prob += (
        lpSum([*cost][i-1] * x[i] for i in x) == B[i] + delta,
        "Total width constraint of panels for a wall [i]",
    )
    # Решаем задачу
    results = prob.solve()
    # Добавляем результаты в словарь
    total_cost += prob.objective.value()
    result_dict[A[i]] = {}
    # Умножаем на 2, для подсчета с двух сторон
    result_dict[A[i]]['Cost'] = prob.objective.value() * 2
    for var in prob.variables():
        result_dict[A[i]][var.name] = var.value() * 2
        

# Выводим результаты в виде таблицы
result_table = pd.DataFrame(result_dict).T
result_table.index.name = 'Wall'
result_table.loc['Total']= result_table.sum()
print("Результаты для зоны B\n")
print(result_table)

Результаты для зоны B

           Cost  300  400  450  500  750   900  delta
Wall                                                 
5650    29000.0  0.0  0.0  0.0  0.0  2.0  10.0  100.0
3800    27000.0  0.0  0.0  0.0  4.0  0.0   8.0    0.0
5800    24500.0  0.0  0.0  0.0  0.0  0.0  10.0  200.0
4450    32800.0  0.0  2.0  0.0  0.0  0.0  12.0    0.0
Total  113300.0  0.0  2.0  0.0  4.0  2.0  40.0  300.0
